<a id="setup"></a>
# <p style="background-color: #ff6200; font-family:calibri; color:white; font-size:120%; font-family:Verdana; text-align:center; border-radius:15px 50px;">Step 1 | Setup and Initialization</p>

<div style="border-radius:10px; padding: 15px; font-size:100%; text-align:left">

<h2 align="left"><font color=#ff6200>Libraries</font></h2>

In [1]:
# Data Wrangling
import polars as pl
import polars.selectors as cs

# Data Pre-Processing
from sklearn.preprocessing import LabelEncoder

# xgboost
import xgboost as xgb

# Metrics
from sklearn.metrics import mean_squared_error

<div style="border-radius:10px; padding: 15px;font-size:100%; text-align:left">

<h2 align="left"><font color=#ff6200>Dataset Description</font></h2>

| __Variable__   | __Description__ |
|     :---       |       :---      |      
| __ID__  | The identifier. |
| __brand__  | The brand of the car. |
| __model__| The model of the car. |
| __model_year__   | The year of the model. |
| __milage__| How many miles the car have. |
| __fuel_type__  | The type of fuel. |
| __engine__ | Engine details. |
| __transmission__    |Transmission details. |
| __ext_col__    | Exterior color. |
| __int_col__    | Interior color. |
| __accident__    | The car has an accident. (1 if yes) |
| __clean_title__    | Clean Tittle. |
| __price__    | The price of car. (Target Variable) |


In [2]:
df = pl.read_csv("train.csv")

<a id="setup"></a>
# <p style="background-color: #ff6200; font-family:calibri; color:white; font-size:120%; font-family:Verdana; text-align:center; border-radius:15px 50px;">Step 2 | Data Wrangling</p>


In [3]:
df.head()

id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
i64,str,str,i64,i64,str,str,str,str,str,str,str,i64
0,"""MINI""","""Cooper S Base""",2007,213000,"""Gasoline""","""172.0HP 1.6L 4 Cylinder Engine…","""A/T""","""Yellow""","""Gray""","""None reported""","""Yes""",4200
1,"""Lincoln""","""LS V8""",2002,143250,"""Gasoline""","""252.0HP 3.9L 8 Cylinder Engine…","""A/T""","""Silver""","""Beige""","""At least 1 accident or damage …","""Yes""",4999
2,"""Chevrolet""","""Silverado 2500 LT""",2002,136731,"""E85 Flex Fuel""","""320.0HP 5.3L 8 Cylinder Engine…","""A/T""","""Blue""","""Gray""","""None reported""","""Yes""",13900
3,"""Genesis""","""G90 5.0 Ultimate""",2017,19500,"""Gasoline""","""420.0HP 5.0L 8 Cylinder Engine…","""Transmission w/Dual Shift Mode""","""Black""","""Black""","""None reported""","""Yes""",45000
4,"""Mercedes-Benz""","""Metris Base""",2021,7388,"""Gasoline""","""208.0HP 2.0L 4 Cylinder Engine…","""7-Speed A/T""","""Black""","""Beige""","""None reported""","""Yes""",97500


<div style="border-radius:10px; padding: 15px; font-size:100%; text-align:left">

<h2 align="left"><font color=#ff6200>String Variables</font></h2>

In [4]:
(
    df.select(cs.by_dtype(pl.String))
    .head(5)
)

brand,model,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
str,str,str,str,str,str,str,str,str
"""MINI""","""Cooper S Base""","""Gasoline""","""172.0HP 1.6L 4 Cylinder Engine…","""A/T""","""Yellow""","""Gray""","""None reported""","""Yes"""
"""Lincoln""","""LS V8""","""Gasoline""","""252.0HP 3.9L 8 Cylinder Engine…","""A/T""","""Silver""","""Beige""","""At least 1 accident or damage …","""Yes"""
"""Chevrolet""","""Silverado 2500 LT""","""E85 Flex Fuel""","""320.0HP 5.3L 8 Cylinder Engine…","""A/T""","""Blue""","""Gray""","""None reported""","""Yes"""
"""Genesis""","""G90 5.0 Ultimate""","""Gasoline""","""420.0HP 5.0L 8 Cylinder Engine…","""Transmission w/Dual Shift Mode""","""Black""","""Black""","""None reported""","""Yes"""
"""Mercedes-Benz""","""Metris Base""","""Gasoline""","""208.0HP 2.0L 4 Cylinder Engine…","""7-Speed A/T""","""Black""","""Beige""","""None reported""","""Yes"""


In [5]:
(
    df
    .select(cs.by_dtype(pl.String))
    .unpivot(variable_name="column")
    .group_by("column")
    .agg(
        Null_Values = (pl.col.value.is_null().sum() / pl.col.value.len()) * 100,
        Unique_Values = (pl.col.value.n_unique())
    )
)

column,Null_Values,Unique_Values
str,f64,u32
"""ext_col""",0.0,319
"""engine""",0.0,1117
"""fuel_type""",2.69608,8
"""model""",0.0,1897
"""int_col""",0.0,156
"""brand""",0.0,57
"""accident""",1.300568,3
"""transmission""",0.0,52
"""clean_title""",11.360876,2


<div style="border-radius: 15px; padding: 20px; background-color: #ffeacc; font-size: 100%; text-align: left; border: 2px solid #ff6200; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
    <h2 style="font-family: Arial, sans-serif; color: #ff6200; margin-bottom: 10px;">String Variables</h2>
    <ul style="font-family: Arial, sans-serif; line-height: 1.8; color: #333; padding-left: 20px;">
        <li><strong>Missing Values:</strong> Missing data is found in 3 variables: <em>"fuel_type"</em> (2.69%), <em>"clean_title"</em> (11.3%), and <em>"accident"</em> (1.3%). The <em>"clean_title"</em> variable has a significant portion of missing values, which I will address later in the process.
        </li>
        <li><strong>Unique Values:</strong> The dataset contains a high number of unique values, particularly in the <em>"model"</em> column, with 1,897 distinct entries.</li>        
        <li><strong>Brand:</strong> There are 57 different car brands in the dataset. I will categorize these into broader groups such as luxury cars, premium cars, mainstream brands, and others.</li>
        <li><strong>Fuel Type:</strong> The <em>"fuel_type"</em> column has 8 different values. To simplify the analysis, I will condense these into 5 categories: 1) Diesel, 2) Hybrid, 3) Electric, 4) Gasoline, and 5) Other.</li>
        <li><strong>Interior and Exterior Color:</strong> Although there are many color options, I will focus on the base colors and create a new column labeled "Special Color" to flag any unique color variations.</li>
        <li><strong>Engine:</strong> The <em>"engine"</em> column contains valuable information such as horsepower (HP), engine displacement (liters), and cylinder count.</li>
        <li><strong>Transmission:</strong> The <em>"transmission"</em> column provides interesting details. I will perform additional data wrangling to extract and organize the key information from this column.</li>
        <li><strong>Model:</strong> The <em>"Model"</em> column have multiple values, so on this first go, i'm going to ignore it.</li>
    </ul>
</div>


<div style="border-radius:10px; padding: 15px; font-size:100%; text-align:left">
<h3 align="left"><font color=#ff6200>Brand</font></h3>
</div>

As i said, im going to create some features for the brand

In [6]:
# Brand
Luxury_list = ["Lambo", "Ferrari", "Bugatti", "Rolls-Royce", "Maserati", "McLaren", "Bentley", "Aston", "Lamborghini", "Maybach"]
Premium_list = ["Mercedes-Benz", "Audi", "BMW", "Porsche", "MINI", "Cadillac","Lexus", "Tesla", "INFINITI",
                "Land", "Lincoln", "Volvo", "Acura", "Jeep", "Honda", "Chrysler", "Genesis", "Pontiac", "Jaguar",
                "Hummer", "Alfa", "Dodge", "Subaru", "FIAT"]
Normal_list = ["Toyota", "Nissan", "Volkswagen", "Hyundai", "Kia", "Chevrolet", "Ford", "GMC", "Mazda", "RAM", "Mitsubishi", "Suzuki"]


def set_brand(brand_col: pl.Expr) -> pl.Expr:
    return (
        pl.when(brand_col.is_in(Luxury_list)).then(3)
        .when(brand_col.is_in(Premium_list)).then(2)
        .when(brand_col.is_in(Normal_list)).then(1)
        .otherwise(0)
        .alias("Brand Type")
    )


<div style="border-radius:10px; padding: 15px; font-size:100%; text-align:left">
<h3 align="left"><font color=#ff6200>Fuel</font></h3>
</div>

Im going to create groups for the Fuel, if the value its null im going to asume it's some kind of gasoline

In [7]:
def set_fuel_type(fuel_type_col: pl.Expr) -> pl.Expr:
    return (
        pl.when(fuel_type_col.is_in(["E85 Flex Fuel", "Gasoline"])).then(1)
        .when(fuel_type_col == "Diesel").then(2)
        .when(fuel_type_col.is_in([ "Hybrid","Plug-In Hybrid" ])).then(3)
        .when(fuel_type_col.is_in([ "not supported"])).then(4)
        .when(fuel_type_col.is_in(["–"])).then(5)
        .otherwise(1)
        .alias("Fuel Type")
    )

<div style="border-radius:10px; padding: 15px; font-size:100%; text-align:left">
<h3 align="left"><font color=#ff6200>Engine</font></h3>
</div>
Going to extract the info that have the engine column

In [8]:
def get_engine() -> pl.Expr:
    return (
        pl.col("engine").str.extract(r'(\d+)\.\d+HP',1).cast(pl.Int64).alias("HP"),
        pl.col("engine").str.extract(r'(\d+\.\d+)L',1).cast(pl.Float32).alias("Displacement"),
        pl.col("engine").str.extract(r'(\d+)\sCylinder',1).cast(pl.Int64).alias("Cylinder"),
        pl.col("engine").str.contains("Electric").alias("Electric")
    )

<div style="border-radius:10px; padding: 15px; font-size:100%; text-align:left">
<h3 align="left"><font color=#ff6200>Color</font></h3>
</div>

Delete any white spaces, and lowercase all the colors, later, transform them into simplier colors

In [9]:
color_list = ["yellow","white", "silver", "black", "blue", "gray", "red", "green", "beige","purple", "orange", "brown", "other"]
pattern = "|".join(color_list)

replace_colors = {
    "ebony" : "green",
    "gold": "yellow",
    "metallic" : "gray",
    "grey" : "gray",
    "verde": "green",
    "caviar" : "black",
    "pearl" : "white",
    "obsidian": "purple",
    "sea" : "blue",
    "blu" : "blue",
    "–": "other"
} 


def get_colors_features()-> pl.Expr:
    return(
        pl.col("ext_col").str.strip_chars().str.to_lowercase().str.replace_many(replace_colors).str.extract(f"({pattern})").alias("ext_color_f").fill_null(pl.lit("other")),
        pl.col("int_col").str.strip_chars().str.to_lowercase().str.replace_many(replace_colors).str.extract(f"({pattern})").alias("int_col_f").fill_null(pl.lit("other"))
    )

def is_special_color() -> pl.Expr:
    return (
        pl
        .when(
            (pl.col("ext_col").str.strip_chars().str.to_lowercase().str.len_chars() > 7) | ~(pl.col("ext_col").str.strip_chars().str.to_lowercase().is_in(color_list))
            )
            .then(1)
            .otherwise(0)
        .alias("Special ext Color")
    )

<div style="border-radius:10px; padding: 15px; font-size:100%; text-align:left">
<h3 align="left"><font color=#ff6200>Transmission</font></h3>
</div>

On the transmission, im going to extract if the car have Manual, aumatic or CVT.

In [10]:
def set_transmission() -> pl.Expr:
    return (
        pl
        .when(pl.col("transmission").str.contains("Automatic")).then(pl.lit("Automatic"))
        .when(pl.col("transmission").str.contains("Manual")).then(pl.lit("Manual"))
        .when(pl.col("transmission").str.contains("A/T")).then(pl.lit("Automatic"))
        .when(pl.col("transmission").str.contains("AT")).then(pl.lit("Automatic"))
        .when(pl.col("transmission").str.contains("Mt")).then(pl.lit("Manual"))
        .when(pl.col("transmission").str.contains("CVT")).then(pl.lit("CVT"))
        .alias("Transmission_processed")
    )

In [11]:
def fill_transmission() -> pl.Expr:
    return (
    pl.col("Transmission_processed").fill_null(pl.col("Transmission_processed").mode().first().over(["brand", "model_year"])).fill_null(pl.lit("Manual"))
    )

<div style="border-radius:10px; padding: 15px; font-size:100%; text-align:left">
<h3 align="left"><font color=#ff6200>Accident</font></h3>
</div>

I will convert the data from string to numeric. Additionally, I will assume that every null value indicates no accident, though this may not apply in all cases."

In [12]:
def set_accident() -> pl.Expr:
    return (
        pl
        .when(pl.col("accident") == "None reported").then(0)
        .when(pl.col("accident") == "At least 1 accident or damage").then(1)
        .otherwise(0)
        .alias("accident")
    )

<div style="border-radius:10px; padding: 15px; font-size:100%; text-align:left">
<h3 align="left"><font color=#ff6200>Clean Tittle</font></h3>
Clean Tittle only have Yes and Null values, so im going to fill the nulls with "No"
</div>

In [13]:
def set_clean_title() -> pl.Expr:
    return (
        pl.col("clean_title").fill_null(pl.lit("No")).str.replace("Yes",1).str.replace("No",0).cast(pl.Int8)
    )

<div style="border-radius:10px; padding: 15px; font-size:100%; text-align:left">
<h2 align="left"><font color=#ff6200>Numeric Variables</font></h2>
</div>

In [14]:
(
    df.select(cs.numeric())
    .head(5)
)

id,model_year,milage,price
i64,i64,i64,i64
0,2007,213000,4200
1,2002,143250,4999
2,2002,136731,13900
3,2017,19500,45000
4,2021,7388,97500


In [15]:
(
    df
    .select(cs.numeric())
    .unpivot(variable_name="column")
    .group_by("column")
    .agg(
        Null_Values = (pl.col.value.is_null().sum() / pl.col.value.len()) * 100,
        Unique_Values = (pl.col.value.n_unique())
    )
)

column,Null_Values,Unique_Values
str,f64,u32
"""model_year""",0.0,34
"""milage""",0.0,6651
"""id""",0.0,188533
"""price""",0.0,1569


<div style="border-radius: 15px; padding: 20px; background-color: #ffeacc; font-size: 100%; text-align: left; border: 2px solid #ff6200; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
    <h2 style="font-family: Arial, sans-serif; color: #ff6200; margin-bottom: 10px;">Numeric Variables</h2>
    <ul style="font-family: Arial, sans-serif; line-height: 1.8; color: #333; padding-left: 20px;">
        <li><strong>Missing Values:</strong> No missing data for numeric variables.</li>
        <li><strong>ID:</strong> No changes needed, as it serves as a unique identifier.</li>
        <li><strong>Price:</strong> The target variable. We'll assume the prices are correct.</li>
        <li><strong>Mileage:</strong> Typically, higher mileage results in a lower price.</li>
        <li><strong>Model Year:</strong> I will create a feature to account for older vehicles. If a vehicle is very old, it might be considered a classic rather than just an old car.</li>
    </ul>
</div>


In [16]:
def categorize_year() -> pl.Expr:
    return (
        pl
        .when((pl.col("model_year") > 1970) & (pl.col("model_year") <= 1980)).then(1)
        .when((pl.col("model_year") > 1981) & (pl.col("model_year") <= 1990)).then(2) 
        .when((pl.col("model_year") > 1991) & (pl.col("model_year") <= 2000)).then(3) 
        .when((pl.col("model_year") > 2001) & (pl.col("model_year") <= 2010)).then(4)
        .when((pl.col("model_year") > 2010) & (pl.col("model_year") <= 2020)).then(5)  
        .otherwise(6)
        .alias("model_year") 
    )

In [17]:
def add_year() -> pl.Expr:
    return (
        (2024 - pl.col("model_year")).alias("Years")
    )

<div style="border-radius:10px; padding: 15px; font-size:100%; text-align:left">
<h2 align="left"><font color=#ff6200>Apply Functions</font></h2>
Lets Apply all the functions together!
</div>

In [18]:
df_with_functions = (
    df
    # String Functions

    .with_columns(set_brand(pl.col("brand")).alias("Brand Type"))
    .with_columns(set_fuel_type(pl.col("fuel_type")).alias("Fuel Type"))
    .with_columns(pl.col("engine").str.replace(r'\sLiter', "L")) # Fix the litter before aplying the engine
    .with_columns(get_engine())
    .with_columns(get_colors_features())
    .with_columns(is_special_color())
    .with_columns(set_transmission())
    .with_columns(fill_transmission())
    .with_columns(set_accident())
    .with_columns(set_clean_title())

    # Numeric Functions
    .with_columns(categorize_year())
    .with_columns(add_year())

)

<div style="border-radius:10px; padding: 15px; font-size:100%; text-align:left">
<h2 align="left"><font color=#ff6200>Dealing With Nulls</font></h2>
Going to fill the null with the mode, over the brand and model year.
</div>

In [19]:
df_filled = (
    df_with_functions
    .with_columns(
        pl.col("HP").fill_null(pl.col("HP").mode().first().over(["brand", "model_year"])).fill_null(pl.col("HP").mean()),
        pl.col("Displacement").fill_null(pl.col("Displacement").mode().first().over(["brand", "model_year"])).fill_null(pl.col("Displacement").mean()),
        pl.col("fuel_type").fill_null(pl.col("fuel_type").mode().first().over(["brand", "model_year"])).fill_null(pl.col("fuel_type").mode()),
        pl.col("Cylinder").fill_null(pl.col("Cylinder").mode().first().over(["brand", "model_year"])).fill_null(pl.col("Cylinder").mode())
    )
)


<div style="border-radius:10px; padding: 15px; font-size:100%; text-align:left">
<h2 align="left"><font color=#ff6200>Selecting and processing the columns</font></h2>
Going to remove the columns that are already processes, and exclude the model and the brand.
</div>

In [20]:
df_to_process = (
    df_filled
    .select(pl.all().exclude(["model","brand", "fuel_type","engine","ext_col", "int_col", "transmission"]))
)

In [21]:
ext_color_enconder = LabelEncoder()
int_color_enconder = LabelEncoder()
transmission_encoder = LabelEncoder()


df_processed = (
    df_to_process
    .with_columns(
        pl.col("ext_color_f").map_batches(ext_color_enconder.fit_transform).alias("ext_color_f_encoded"),
        pl.col("int_col_f").map_batches(int_color_enconder.fit_transform).alias("int_col_f_encoded"),
        pl.col("Transmission_processed").map_batches(transmission_encoder.fit_transform).alias("Transmission_processed_encoded"),
        pl.col("Electric").cast(pl.Int8)
    )
    .select(
        pl.all().exclude(["ext_color_f","int_col_f","Transmission_processed"])
    )
)

<div style="border-radius:10px; padding: 15px; font-size:100%; text-align:left">
<h2 align="left"><font color=#ff6200>Machine learning</font></h2>
The part you were expecting, let try some model to predict the price
</div>

In [22]:
model = xgb.XGBRegressor()

model.fit(
    X = df_processed.select(pl.all().exclude(["id", "price"])),
    y = df_processed["price"])

# Make predictions
predictions = model.predict(df_processed.select(pl.all().exclude(["id", "price"])))

# Calculate RMSE
rmse = mean_squared_error(y_true=df_processed["price"], y_pred=predictions, squared=False)
print(f"RMSE: {rmse}")

# Add the predictions to the original DataFrame
predictions_df = pl.DataFrame({"pos": predictions})
result_df = pl.concat([df_processed, predictions_df], how="horizontal")

RMSE: 63772.01652505908


c:\Users\Arancium\Proyectos\env\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [23]:
result_df

id,model_year,milage,accident,clean_title,price,Brand Type,Fuel Type,HP,Displacement,Cylinder,Electric,Special ext Color,Years,ext_color_f_encoded,int_col_f_encoded,Transmission_processed_encoded,pos
i64,i32,i64,i32,i8,i64,i32,i32,f64,f32,i64,i8,i32,i32,i64,i64,i64,f32
0,4,213000,0,1,4200,2,1,172.0,1.6,4,0,0,2020,12,4,0,6988.742676
1,4,143250,0,1,4999,2,1,252.0,3.9,8,0,0,2020,10,0,0,12823.620117
2,4,136731,0,1,13900,1,1,320.0,5.3,8,0,0,2020,2,4,0,11612.020508
3,5,19500,0,1,45000,2,1,420.0,5.0,8,0,0,2019,1,1,0,68928.945312
4,6,7388,0,1,97500,2,1,208.0,2.0,4,0,0,2018,1,0,0,77681.320312
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
188528,5,49000,0,1,27500,2,1,420.0,6.2,8,0,0,2019,11,0,0,46424.316406
188529,5,28600,0,1,30000,2,1,385.0,3.0,6,0,0,2019,11,1,0,61110.089844
188530,6,13650,0,1,86900,2,1,469.0,4.0,8,0,0,2018,11,1,0,97781.546875


In [24]:
data_test = (pl.read_csv("test.csv"))

In [25]:
df_test_processed  = (
    data_test
    # String Functions

    .with_columns(set_brand(pl.col("brand")).alias("Brand Type"))
    .with_columns(set_fuel_type(pl.col("fuel_type")).alias("Fuel Type"))
    .with_columns(pl.col("engine").str.replace(r'\sLiter', "L")) # Fix the litter before aplying the engine
    .with_columns(get_engine())
    .with_columns(get_colors_features())
    .with_columns(is_special_color())
    .with_columns(set_transmission())
    .with_columns(fill_transmission())
    .with_columns(set_accident())
    .with_columns(set_clean_title())

    # Numeric Functions
    .with_columns(categorize_year())
    .with_columns(add_year())

    
    .with_columns(
        pl.col("HP").fill_null(df_processed.select(pl.col("HP").mean())),
        pl.col("Displacement").fill_null(df_processed.select(pl.col("Displacement").mean())),
        pl.col("Fuel Type").fill_null(df_processed.select(pl.col("Fuel Type").mean())),
        pl.col("Cylinder").fill_null(df_processed.select(pl.col("Cylinder").mean()))
    )

    .select(pl.all().exclude(["model","brand", "fuel_type","engine","ext_col", "int_col", "transmission"]))
    .with_columns(
        pl.col("ext_color_f").map_batches(ext_color_enconder.transform).alias("ext_color_f_encoded"),
        pl.col("int_col_f").map_batches(int_color_enconder.transform).alias("int_col_f_encoded"),
        pl.col("Transmission_processed").map_batches(transmission_encoder.transform).alias("Transmission_processed_encoded"),
        pl.col("Electric").cast(pl.Int8)
    )
    .select(
        pl.all().exclude(["ext_color_f","int_col_f","Transmission_processed"])
    )
    
)

In [26]:
predictions_test = model.predict(df_test_processed.select(pl.all().exclude(["id", "price"])))
predictions_df = pl.DataFrame({"price": predictions_test})

result_df = pl.concat([df_test_processed, predictions_df], how="horizontal")

(
    result_df
    .select(
        pl.col("id"),
        pl.col("price")
    )
    .write_csv("submission.csv")
)

<div style="border-radius: 15px; padding: 20px; background-color: #ffeacc; font-size: 100%; text-align: left; border: 2px solid #ff6200; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);"> <h2 style="font-family: Arial, sans-serif; color: #ff6200; margin-bottom: 10px;">Next Steps</h2> <ul style="font-family: Arial, sans-serif; line-height: 1.8; color: #333; padding-left: 20px;"> <li>Experiment with a wider range of machine learning models to improve performance.</li> <li>Utilize cross-validation with GridSearchCV for hyperparameter tuning.</li> <li>Implement ensemble learning techniques to combine multiple models.</li> <li>Explore alternative data preprocessing methods to enhance data quality.</li> </ul>
</div>
Thanks everyone for reading the Kernel! 